In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import CLIPTokenizer, CLIPTextModel, CLIPVisionModel
from torch.utils.data import DataLoader, Dataset
import PIL
import os
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms
import regex
import numpy as np
from sklearn.model_selection import train_test_split
import random
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [30]:
def comp_type_map(comp_type):
    type_to_label = {'Direct': 0, 'Metaphorical': 1, 'Semantic list': 2, 'Reduplication': 3, 'Single': 4}
    return type_to_label[comp_type]

def label_to_comp_type(label):
    label_to_type = {0: 'Direct', 1: 'Metaphorical', 2: 'Semantic list', 3: 'Reduplication', 4: 'Single'}
    return label_to_type[label]

In [62]:
class EmojiCLIP(nn.Module):
    def __init__(self, vit_model=CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")):
        super().__init__()
        self.vit = vit_model

        # freeze ViT in early training
        for param in self.vit.parameters():
            param.requires_grad = False
    def extract_embedding(self, image):
        with torch.no_grad():
            return self.vit(pixel_values=image).pooler_output
    
    def forward(self, image):
        image_embedding = self.extract_embedding(image)
        return image_embedding
    
class EmojiImageDataset(Dataset):
    def __init__(self, image_dir, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])):
        super().__init__()
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.images[idx])
        image = PIL.Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
emoji_image_dataset = EmojiImageDataset('combine_em_images')
model = EmojiCLIP().to(device)
all_embeddings = []
for i, image_embed in tqdm(enumerate(emoji_image_dataset), desc='Extracting embeddings', total=len(emoji_image_dataset)):
        batch_embeddings = model(image_embed.unsqueeze(0).to(device))
        all_embeddings.extend(batch_embeddings)

Extracting embeddings: 100%|██████████| 1655/1655 [04:18<00:00,  6.40it/s]


In [73]:
class EmojisDataset(Dataset):
    def __init__(self, elco_df, image_embeds, em_max_len=3, text_max_len=6, tokenizer='openai/clip-vit-base-patch32'):
        super().__init__()
        self.transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
        self.elco_df = elco_df
        self.image_embeds = image_embeds
        self.emoji_descriptions = [self.preprocess_emoji_description(desc) for desc in elco_df["Description"]]
        self.raw_emoji_descriptions = elco_df["Description"].values
        self.em_max_len = em_max_len
        self.text_max_len = text_max_len
        self.clip_tokenizer = CLIPTokenizer.from_pretrained(tokenizer)
        self.composition_type = [comp_type_map(t) for t in elco_df['Composition strategy'].values]
    
    def preprocess_emoji_description(self, text):
        text = text.replace('\'\'', '').lower()
        split_text = regex.findall(r'\':?(.*?):?\'', text)
        return split_text
    def preprocess_en(self, text):
        return "This is" + text.lower().strip()
    
    def __len__(self):
        return len(self.emoji_descriptions)
    
    def __getitem__(self, index):
        emoji_descs = self.emoji_descriptions[index]
        image_embed = self.image_embeds[index]
        en_text = self.preprocess_en(self.elco_df['EN'].values[index])
        composition_type = self.composition_type[index]
        
        
        emoji_desc = " ".join(emoji_descs)
        desc_tokens = self.clip_tokenizer(emoji_desc, 
                                  truncation=True, 
                                  padding='max_length',
                                  return_tensors='pt',
                                  max_length=64)
        
        en_tokens = self.clip_tokenizer(en_text, 
                                  truncation=True, 
                                  padding='max_length',
                                  return_tensors='pt',
                                  max_length=self.text_max_len)
        en_tokens = {k: v.squeeze(0) for k, v in en_tokens.items()}
        desc_tokens = {k: v.squeeze(0) for k, v in desc_tokens.items()}
        return (image_embed, desc_tokens, en_tokens), composition_type
        

In [32]:
class TypeClassifier(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=512, num_classes=5, dropout=0.1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes),
        )
        
    def forward(self, x):
        return self.model(x)

In [78]:
class EmojiCompositionModel(nn.Module):
    def __init__(self, text_encoder='openai/clip-vit-base-patch32',
                 image_embed_dim=768, projection_dim=512, freeze_clip=False, text_ratio=0.5, visual_ratio=0.5):
        super().__init__()


        # self.emoji_transformer = ImageEmbedTransformer(embedding_dim=image_embed_dim, num_layers=num_layers)

        self.text_encoder = CLIPTextModel.from_pretrained(text_encoder).text_model
        self.text_hidden_dim = self.text_encoder.config.hidden_size
        self.desc_embed_size = int(projection_dim * text_ratio)
        self.image_embed_size = int(projection_dim * visual_ratio)
        self.text_proj_dim = self.desc_embed_size + self.image_embed_size
        
        if freeze_clip:
            for name, param in self.text_encoder.named_parameters():
                    # if "encoder.layers.10" in name or "encoder.layers.11" in name:
                    #     param.requires_grad = True
                    # else:
                    param.requires_grad = False
                    
        
        self.text_proj = nn.Sequential(
            nn.Linear(self.text_hidden_dim, self.text_proj_dim),
            nn.ReLU(),
            nn.LayerNorm(self.text_proj_dim),
            nn.Dropout(0.1),
            )
        
        self.desc_proj = nn.Sequential(
            nn.Linear(self.text_hidden_dim, self.desc_embed_size),
            nn.ReLU(),
            nn.LayerNorm(self.desc_embed_size),
            nn.Dropout(0.1),
            )
        
        self.image_proj = nn.Sequential(
            nn.Linear(image_embed_dim, self.image_embed_size),
            nn.ReLU(),
            nn.LayerNorm(self.image_embed_size),
            nn.Dropout(0.1)
            )
        
        input_dim = self.text_proj_dim * 3
        self.classifier = TypeClassifier(input_dim=input_dim, hidden_dim=projection_dim * 2, num_classes=5)
        
    def forward(self, image_embed, desc_input, text_input):
        desc_out = self.text_encoder(**desc_input).pooler_output
        z_desc = self.desc_proj(desc_out)
        z_image = self.image_proj(image_embed)
        
        z_emoji = torch.concatenate([z_desc, z_image], dim=-1)
        
        text_out = self.text_encoder(**text_input).pooler_output
        z_text = self.text_proj(text_out)

        z_emoji = F.normalize(z_emoji, dim=-1)
        z_text = F.normalize(z_text, dim=-1)
        z_concat = torch.cat([z_emoji, z_text, torch.abs(z_emoji - z_text)], dim=-1)
        output = self.classifier(z_concat)

        return (z_emoji, z_text), output

In [34]:
elco_df = pd.read_csv('../../data/ELCo.csv')

In [74]:
emojis_train_dataset = EmojisDataset(elco_df, all_embeddings)
train_dataset, validate_dataset = train_test_split(emojis_train_dataset, test_size=0.08, shuffle=True, random_state=42)
emojis_train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
emojis_validate_dataloader = DataLoader(validate_dataset, batch_size=32, shuffle=False, drop_last=True)
# emojis_test_dataset = EmojisDataset(emoji_embed_dict, test_df)
# emojis_test_dataloader = DataLoader(emojis_test_dataset, batch_size=32, shuffle=False, drop_last=True)

In [36]:
class EarlyStopping:
    def __init__(self, patience=2, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss, accuracy):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
def train(text_ratio, visual_ratio, verbose=False):
    # reconstruct text dataloader -> each bacth is a list of all possible en for the emojis in given batch
    labels = [data[1] for data in emojis_train_dataset]
    class_counts = Counter(labels)
    class_weights = torch.tensor([len(labels) / count for label, count in class_counts.items()])

    en_em_model = EmojiCompositionModel(freeze_clip=True, text_ratio=text_ratio, visual_ratio=visual_ratio).to(device)
    optimizer = torch.optim.AdamW(en_em_model.parameters(), lr=2e-5)
    early_stopping = EarlyStopping(patience=3)
    # criterion = nn.CrossEntropyLoss(weight=class_weights)
    num_epochs = 45
    results = []

    for epoch in tqdm(range(num_epochs), total=num_epochs, desc=f'Training at p_text={text_ratio}'):
        total_loss = 0.0
        en_em_model.train()
        incorrect_labels = {'Direct': 0, 'Metaphorical': 0, 'Semantic list': 0, 'Reduplication': 0, 'Single': 0}
        correct_count_train = 0
        
        for (image, desc_input, en_input), ctype in emojis_train_dataloader:
            (z_emojis, z_text), output = en_em_model(image, desc_input, en_input)
            
            classifier_loss = F.cross_entropy(output, ctype, weight=class_weights, reduction='none')
            
            classifier_pred = torch.argmax(output, dim=-1)
            correct_count_train += torch.sum(classifier_pred == ctype).item()
            # for i, pred in enumerate(clssifier_pred):
            #     if pred != ctype[i]:
            #         incorrect_labels[label_to_comp_type(ctype[i].item())] += 1
            
            
            target_class = torch.tensor([0, 1])
            is_target_class = torch.isin(ctype, target_class)
            is_incorrect = (classifier_pred != ctype)
            should_double = is_target_class & is_incorrect
            # print(classifier_loss,"classifier_loss")
            weights = torch.ones_like(classifier_loss)
            weights[should_double] = 2
            adjusted_loss = classifier_loss * weights
            classifier_loss = adjusted_loss.mean()
                    
            match_labels = torch.arange(emojis_train_dataloader.batch_size).to(device)
            loss_per_emojis = z_emojis @ z_text.T
            loss_per_text = loss_per_emojis.T
            loss_em = F.cross_entropy(loss_per_emojis, match_labels)
            loss_text = F.cross_entropy(loss_per_text, match_labels)

            loss = classifier_loss + (loss_em + loss_text) / 2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(emojis_train_dataloader)
        train_accuracy = correct_count_train / len(emojis_train_dataset)
            
        en_em_model.eval()
        correct_count = 0
        precision_scores = []
        recall_scores = []
        f1_scores = []
        with torch.no_grad():
            for (image, desc_input, en_input), ctype in emojis_validate_dataloader:
                _, output = en_em_model(image, desc_input, en_input)
                prediction = torch.argmax(output, dim=-1)
                correct_count += torch.sum(prediction == ctype).item()
                precision_scores.append(precision_score(ctype.cpu(), prediction.cpu(), average='macro', zero_division=0))
                recall_scores.append(recall_score(ctype.cpu(), prediction.cpu(), average='macro', zero_division=0))
                f1_scores.append(f1_score(ctype.cpu(), prediction.cpu(), average='macro', zero_division=0))
                for i, pred in enumerate(prediction):
                    # print(f"Pred: {pred}, True: {ctype[i]}")
                    if pred != ctype[i]:
                        incorrect_labels[label_to_comp_type(ctype[i].item())] += 1
            validation_accuracy = correct_count / len(emojis_validate_dataset)
            precision = np.mean(precision_scores)
            recall = np.mean(recall_scores)
            f1 = np.mean(f1_scores)
            
        if verbose:    
            print(f'Incorrect labels: {incorrect_labels}')
            print(f'Epoch {epoch} - Train Loss: {total_loss}, Train accuracy: {train_accuracy}')
            print(f'Validation Accuracy: {validation_accuracy}, recall: {recall}, precision: {precision}, f1: {f1}')
            print("=============================")
        result = {
                "train_accuracy": train_accuracy,
                "validation_accuracy": validation_accuracy,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "incorrect_labels": incorrect_labels,
            }
        results.append(result)
        
        early_stopping(total_loss, validation_accuracy)
        if early_stopping.early_stop:
            if verbose:
                print(f'Early stopping at epoch {epoch}')
            break
    return results, en_em_model

In [82]:
train(0.5, 0.5, verbose=True)

Training at p_text=0.5:   2%|▏         | 1/45 [00:51<37:38, 51.33s/it]

Incorrect labels: {'Direct': 7, 'Metaphorical': 27, 'Semantic list': 7, 'Reduplication': 6, 'Single': 7}
Epoch 0 - Train Loss: 13.37894078518482, Train accuracy: 0.3861027190332326
Validation Accuracy: 0.556390977443609, recall: 0.3145299145299145, precision: 0.2671776288726753, f1: 0.27859013184652787


Training at p_text=0.5:   4%|▍         | 2/45 [01:28<30:53, 43.11s/it]

Incorrect labels: {'Direct': 8, 'Metaphorical': 31, 'Semantic list': 7, 'Reduplication': 6, 'Single': 7}
Epoch 1 - Train Loss: 13.142476852903975, Train accuracy: 0.44410876132930516
Validation Accuracy: 0.518796992481203, recall: 0.28976266788766786, precision: 0.24555735930735928, f1: 0.25463244568192095


Training at p_text=0.5:   7%|▋         | 3/45 [02:05<28:15, 40.37s/it]

...

Training at p_text=0.5:  98%|█████████▊| 44/45 [29:33<00:46, 46.64s/it]

Incorrect labels: {'Direct': 15, 'Metaphorical': 30, 'Semantic list': 2, 'Reduplication': 4, 'Single': 3}
Epoch 43 - Train Loss: 4.62785823294457, Train accuracy: 0.8078549848942598
Validation Accuracy: 0.556390977443609, recall: 0.5539797008547008, precision: 0.5388392163392164, f1: 0.5370568669835911


Training at p_text=0.5: 100%|██████████| 45/45 [31:16<00:00, 41.70s/it]

Incorrect labels: {'Direct': 15, 'Metaphorical': 30, 'Semantic list': 2, 'Reduplication': 4, 'Single': 3}
Epoch 44 - Train Loss: 4.608934229992806, Train accuracy: 0.8096676737160121
Validation Accuracy: 0.556390977443609, recall: 0.5539797008547008, precision: 0.5388392163392164, f1: 0.5370568669835911


([{'train_accuracy': 0.3861027190332326,
   'validation_accuracy': 0.556390977443609,
   'recall': 0.3145299145299145,
   'precision': 0.2671776288726753,
   'f1': 0.27859013184652787,
   'incorrect_labels': {'Direct': 7,
    'Metaphorical': 27,
    'Semantic list': 7,
    'Reduplication': 6,
    'Single': 7}},
  {'train_accuracy': 0.44410876132930516,
   'validation_accuracy': 0.518796992481203,
   'recall': 0.28976266788766786,
   'precision': 0.24555735930735928,
   'f1': 0.25463244568192095,
   'incorrect_labels': {'Direct': 8,
    'Metaphorical': 31,
    'Semantic list': 7,
    'Reduplication': 6,
    'Single': 7}},
  {'train_accuracy': 0.43564954682779455,
   'validation_accuracy': 0.5037593984962406,
   'recall': 0.2846535409035409,
   'precision': 0.2437878787878788,
   'f1': 0.24449836497799146,
   'incorrect_labels': {'Direct': 6,
    'Metaphorical': 35,
    'Semantic list': 7,
    'Reduplication': 6,
    'Single': 7}},
  {'train_accuracy': 0.4453172205438066,
   'validation_